## Pre-Process Data ##

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
import sklearn as sk

### Extact, Transform, Load (ETL) ###

In [ ]:
def load_data_imdb_as_df(folder):
    """
    loads pos and neg from either train or test
    
    Returns a Pandas dataframe with:
    Colums = [Index, Word, Frequency, Rating, URL]
    Rows = [int, st, int, int, str]
    """
    colums = ["ID","Word","Frequency","Rating","URL"]
    data = {}
    #TODO: Implement
    pass

def reduce_imdb_data_dimensionality(df):
    """
    Reduce dimensionality of set based on correlations and fequency
    
    """
    pass

def reduce_news_data_dimensionality(df):
    """
    Bring down size of sk learn dataset by:
    - Picking 5 categories
    - Filtering out rare words
    """
    pass



In [ ]:
def evaluate_binary_classification(df):
    """
    use Receiver Operating Characteristic (ROC) curve and area under the ROC curve (AUROC)
    """
    pass

def evaluate_multiclass_classification(df):
    """
    compute classification accuracy
    """
    pass

def plot_binary_classification(df):
    """
    plot ROC curve
    compare to DT from sklearn
    """
    pass

def plot_multiclass_classification(df):
    """
    compare accuracy to DT from sklearn
    
    """
    pass




## Experiments ##

### Experiment 1 ###

In [ ]:
def experiment_one():
    """
    Report the:
    - top 10 features with the most positive coefficients
    - top 10 features with the most negative coefficients 
    
    on the IMDB data using simple linear regression on the movie rating scores
    """
    pass

### Experiment 2 ###

In [ ]:
def experiment_two():
    """
    Implement and conduct:
    - Binary classification on the IMDb Reviews
    - Multi-class classification on the 20 news group dataset
    """
    pass

### Experiment 3 ###

In [ ]:
def experiment_three():
    """
    On the same plot as 2, draw ROC curves and report the AUROC 
    values of logistic regression and Decision Trees on the 
    IMDB data binary classification task
    
    """
    pass

### Experiment 4 ###

In [ ]:
def experiment_four():
    """
    Report the:
    - Multiclass classification accuracy of multiclass linear regression 
    - Decision Trees on the 5 chosen classes from the 20-news-group data
    """
    pass

### Experiment 5 ###

In [ ]:
def experiment_five():
    """
    Plot and compare the accuracy of the two models as a function of the
    size of dataset by controlling the training size
    
    For example, you can randomly select 20%, 40%, 60% and 80% of the available 
    training data and train your model on this subset and evaluate the trained 
    model on the held-out test set.
    """
    pass

### Experiment 6 ###

In [ ]:
def experiment_six():
    """
    Compare and evaluate the performance of different learning rates
    """
    pass

### Experiment 7 ###

In [ ]:
def experiment_seven():
    """
    Evaluate the performance of the multiclass regression on more than 5 classes
    
    Compare the top k (e.g. k =3) predicted classes. 
    A correct prediction is determined by whether the true label is within the top k predicted labels. 
    The scoring mechanism involves assigning a score of 1 if the correct label is among the top k predictions and 0 otherwise.
    """
    pass